This is the initial notebook to explore the data and run the first models. 

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

from fastai.imports import *
from fastai.tabular.all import *

from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import eli5
from eli5.sklearn import PermutationImportance


from scipy.stats import skew, norm
from catboost import CatBoostRegressor
from feature_engine.transformation import YeoJohnsonTransformer
from category_encoders.binary import BinaryEncoder

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import shap
import xgboost
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from mlxtend.regressor import StackingCVRegressor

import optuna
from optuna.samplers import TPESampler

In [3]:
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')

Based on initial analysis the train and test datasets have similar characteristics, so it will be easier to combine them for imputation and data analysis work. 

In [4]:
train_test = pd.concat([train, test], ignore_index=True)

In [5]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
SalePrice       1459
dtype: int64

It looks like Alley, FireplaceQu, PoolQC, Fence and MiscFeature have significant numbers of missing data. So it will be best to eliminate those rows. There are a number of rows that have less than 5 rows with missing data. Since some of these are categorical and some are continuous data, their missing data will be replaced with the most frequent value. 

Instead, for high_nan, replace the NaN with None; since they don't have that feature. 

In [6]:
drop_high_nan=['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
train_test[drop_high_nan] = train_test[drop_high_nan].fillna('None')


small_nan_cols = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                  'TotalBsmtSF', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'GarageCars', 
                  'GarageArea','SaleType', 'SaleCondition']
small_impute = SimpleImputer(strategy='most_frequent')
train_test[small_nan_cols] = pd.DataFrame(small_impute.fit_transform(train_test[small_nan_cols]),columns=small_nan_cols)

The following columns seem to have one value significantly larger than the rest, and it would probably be best to use the mode, or most common, value to feel each NaN value: MasVnrType, MasVnrArea, BsmtCond, BsmtExposure, BsmtFinType2, GarageType, GarageFinish, GarageQual, and GarageCond. That is represents 9 out of the 13 columns. 

BsmtQual has two values larger than the rest: Gd and TA. But it only has 2.8% NaNs, so simply using the mode might be good enough.  

GarageYrBlt has 59 NaNs out 2919 rows which is only 2%. It has a dispersed set of values, so it might be easiest just to have any NaNs have the same value as YearBuilt. 

BsmtFinType1 has only 2.7% value of NaNs, and most two of its largest values are GLQ and Unf. It might be easiest to use the mode here. 

LotFrontage has 486 NaNs out of 2919 rows which is a pretty high 16.7%. It has a dispersed range of values, but looking at its characteristics from the describe function above, it seems to have a pretty even distribution with a mean of 10,168 and a median of 9,453. So using the mean to fill in the NaNs seems reasonable. If it turns out this value has a high impact on the predictability of the SalePrice, then it might be good to revisit this assumption. 

In [7]:
mode_cols = ['MasVnrType', 'MasVnrArea', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'BsmtQual', 'BsmtFinType1']
mode_impute = SimpleImputer(strategy='most_frequent')
train_test[mode_cols] = pd.DataFrame(mode_impute.fit_transform(train_test[mode_cols]),columns=mode_cols)

garage_cols = ['GarageType', 'GarageFinish', 'GarageQual','GarageCond', 'GarageYrBlt']
train_test[garage_cols] = train_test[garage_cols].fillna('None')

train_test['LotFrontage'].fillna((train_test['LotFrontage'].mean()), inplace=True)

In [8]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

SalePrice    1459
dtype: int64

In [9]:
train_test['BsmtQual'].unique()

array(['Gd', 'TA', 'Ex', 'Fa'], dtype=object)

In [10]:
train_test['BsmtFinType1'].unique()

array(['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', 'LwQ'], dtype=object)

In [11]:
train_test.BsmtQual = train_test.BsmtQual.replace({"Ex": 110, "Gd": 95, "TA": 85, "Fa": 75, "Po": 60, "NA": 0})
train_test.BsmtFinType1 = train_test.BsmtFinType1.replace({"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1,
                                                         "NA": 0})

To look at how many features have a skew above 0.6, since high skew can be an issue in regression analysis. 

In [12]:
number_cols = train_test.select_dtypes(include=np.number).columns.tolist()

In [13]:
skew_features = train_test[number_cols].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skew_features[skew_features > 0.6]
skew_index = high_skew.index
skew_index

Index(['MiscVal', 'PoolArea', 'LotArea', 'LowQualFinSF', '3SsnPorch',
       'KitchenAbvGr', 'EnclosedPorch', 'ScreenPorch', 'OpenPorchSF',
       'WoodDeckSF', 'LotFrontage', '1stFlrSF', 'MSSubClass', 'GrLivArea',
       '2ndFlrSF', 'BsmtQual', 'TotRmsAbvGrd', 'Fireplaces', 'HalfBath'],
      dtype='object')

In [14]:
print("There are {} numerical features with Skew > 0.6 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skewness

There are 19 numerical features with Skew > 0.6 :


,Skew
MiscVal,21.947195
PoolArea,16.898328
LotArea,12.822431
LowQualFinSF,12.088761
3SsnPorch,11.376065
KitchenAbvGr,4.302254
EnclosedPorch,4.003891
ScreenPorch,3.946694
OpenPorchSF,2.535114
WoodDeckSF,1.842433


To normalize the features with skew above 0.6 using the Yeo-Johnson transformation where, unlike the Box-Cox transformation, zeros can be utilized. These transformations use several values of transformer parameters and select the one that returns the best transformation is selected. 

In [15]:
yjt = YeoJohnsonTransformer()
yjt.fit(train_test[skew_index])
train_test[skew_index] = yjt.transform(train_test[skew_index])

It will be necessary to identify all the columns that have non-numeric object values and then convert them to numeric values. 

In [16]:
obj_cols = list(train_test.select_dtypes(['object']).columns)

In [17]:
for column in obj_cols:
     train_test[column] = pd.factorize(train_test[column], sort=True)[0]

To create some new columns that might compound the effects of the existing columns with higher impact on the xgb score. Used the get_score function with 'gain' as the importance type. This takes the average gain of the tree splits. 

In [18]:
train_test['QualCondSum'] = train_test['OverallQual'] + train_test['OverallCond']
train_test['RemodTime'] = train_test['YearRemodAdd'] - train_test['YearBuilt']
train_test['BsmtFinTypeSF1'] = train_test['BsmtFinType1'] * train_test['BsmtFinSF1']
train_test['TotalFlrSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF']
train_test['TotalFinSF'] = train_test['GrLivArea'] + train_test['BsmtFinSF1']
train_test['GarageCarArea'] = train_test['GarageArea'] * train_test['GarageCars']
train_test['TotalSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF'] + train_test['TotalBsmtSF']

To create a column with the log of the SalePrice to match the evaluation process in the contest. 

In [19]:
train_test['LogSalePrice'] = train_test['SalePrice'].apply(np.log)

To separate the train_test dataset back into the train and test datasets, identify the independent and dependent columns, and create the validation split.

In [20]:
train = train_test[train_test['SalePrice'].notnull()].copy()
test = train_test[train_test['SalePrice'].isnull()].drop(['SalePrice','LogSalePrice'],axis=1)
X = train.drop(['SalePrice','LogSalePrice'],axis=1)
y = train.LogSalePrice

In [21]:
# Setup cross validation folds
kf = KFold(n_splits=12, random_state=42, shuffle=True)

In [22]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kf))
    return rmse

In [23]:
# Needs to be outside the trial for CatBoost
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
def catboost_objective(trial):
    param = {  
        
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "learning_rate": trial.suggest_float("learning_rate", .001, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg", .0001, 25.0),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "bagging_temperature":trial.suggest_float("bagging_temperature", 0, 10),
        "model_size_reg":trial.suggest_float("model_size_reg", 0.1, 1.0, log=True),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
            }        

    
    model = CatBoostRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    

    return rmse

In [25]:
study_catboost = optuna.create_study(sampler=TPESampler(), direction='minimize')
study_catboost.optimize(catboost_objective, n_trials=1000)
print('Number of finished trials:', len(study_catboost.trials))
print('Best trial:', study_catboost.best_trial.params)

[I 2023-06-16 20:29:39,021] A new study created in memory with name: no-name-b36eab48-9820-4261-b636-c325d9d1d2c2
[I 2023-06-16 20:29:57,049] Trial 0 finished with value: 0.15957977719046926 and parameters: {'colsample_bylevel': 0.021402107218988248, 'learning_rate': 0.013266190634399457, 'depth': 10, 'l2_leaf_reg': 13.603220273948672, 'subsample': 0.1379500928238232, 'bagging_temperature': 1.3721307077561862, 'model_size_reg': 0.9217442865472035, 'boosting_type': 'Ordered'}. Best is trial 0 with value: 0.15957977719046926.
[I 2023-06-16 20:32:45,794] Trial 1 finished with value: 0.14280133062201986 and parameters: {'colsample_bylevel': 0.06645375601223097, 'learning_rate': 0.08858401199397013, 'depth': 11, 'l2_leaf_reg': 18.739068694830955, 'subsample': 0.7048888343346147, 'bagging_temperature': 7.759494636225712, 'model_size_reg': 0.885396878840306, 'boosting_type': 'Ordered'}. Best is trial 1 with value: 0.14280133062201986.
[I 2023-06-16 20:32:51,476] Trial 2 finished with value: 0

[I 2023-06-16 20:34:49,438] Trial 19 finished with value: 0.3177295804015216 and parameters: {'colsample_bylevel': 0.059492629745341155, 'learning_rate': 0.001542343600922487, 'depth': 1, 'l2_leaf_reg': 7.576318129677681, 'subsample': 0.5175164632393435, 'bagging_temperature': 1.9170924918812513, 'model_size_reg': 0.24210592783739812, 'boosting_type': 'Plain'}. Best is trial 16 with value: 0.12714026660905367.
[I 2023-06-16 20:34:50,927] Trial 20 finished with value: 0.12779133880546048 and parameters: {'colsample_bylevel': 0.08800768617107753, 'learning_rate': 0.07251756917205755, 'depth': 4, 'l2_leaf_reg': 11.705913569267643, 'subsample': 0.18813712799299442, 'bagging_temperature': 3.5292256059225573, 'model_size_reg': 0.1325279372345458, 'boosting_type': 'Plain'}. Best is trial 16 with value: 0.12714026660905367.
[I 2023-06-16 20:34:52,597] Trial 21 finished with value: 0.13151183678724762 and parameters: {'colsample_bylevel': 0.09194827089105918, 'learning_rate': 0.0718618008836034

[I 2023-06-16 20:35:32,041] Trial 39 finished with value: 0.12833558182610744 and parameters: {'colsample_bylevel': 0.07168114426936661, 'learning_rate': 0.07455366573321134, 'depth': 3, 'l2_leaf_reg': 2.7807482047857865, 'subsample': 0.44111362040005014, 'bagging_temperature': 1.9726815313424566, 'model_size_reg': 0.18775309650357708, 'boosting_type': 'Plain'}. Best is trial 28 with value: 0.1269486886863682.
[I 2023-06-16 20:35:35,273] Trial 40 finished with value: 0.13205096303032626 and parameters: {'colsample_bylevel': 0.07106940091024011, 'learning_rate': 0.07364480810406895, 'depth': 3, 'l2_leaf_reg': 2.1405573001356264, 'subsample': 0.43993430393995975, 'bagging_temperature': 1.9644647533128703, 'model_size_reg': 0.17369316272875543, 'boosting_type': 'Ordered'}. Best is trial 28 with value: 0.1269486886863682.
[I 2023-06-16 20:35:37,495] Trial 41 finished with value: 0.12893876407632263 and parameters: {'colsample_bylevel': 0.06496534590300712, 'learning_rate': 0.06671727918926

[I 2023-06-16 20:37:42,822] Trial 59 finished with value: 0.1344293544824906 and parameters: {'colsample_bylevel': 0.09516585284813571, 'learning_rate': 0.06329077623498913, 'depth': 6, 'l2_leaf_reg': 9.65281321376543, 'subsample': 0.2817398437224561, 'bagging_temperature': 4.590129888022579, 'model_size_reg': 0.17500981231389653, 'boosting_type': 'Plain'}. Best is trial 28 with value: 0.1269486886863682.
[I 2023-06-16 20:37:47,470] Trial 60 finished with value: 0.13588437077744706 and parameters: {'colsample_bylevel': 0.08284830090778425, 'learning_rate': 0.06184468445801144, 'depth': 7, 'l2_leaf_reg': 6.773930903092817, 'subsample': 0.3162435410296986, 'bagging_temperature': 5.4078835686773274, 'model_size_reg': 0.15519405416089432, 'boosting_type': 'Plain'}. Best is trial 28 with value: 0.1269486886863682.
[I 2023-06-16 20:37:49,744] Trial 61 finished with value: 0.1322284069338343 and parameters: {'colsample_bylevel': 0.06401872632792306, 'learning_rate': 0.07354942962555588, 'dept

[I 2023-06-16 20:38:36,050] Trial 79 finished with value: 0.1323010184392259 and parameters: {'colsample_bylevel': 0.07289654149471447, 'learning_rate': 0.07670246013456172, 'depth': 3, 'l2_leaf_reg': 6.03201766628312, 'subsample': 0.34449331774869757, 'bagging_temperature': 3.4950561339999964, 'model_size_reg': 0.16426275978335578, 'boosting_type': 'Ordered'}. Best is trial 74 with value: 0.12640867299816086.
[I 2023-06-16 20:38:38,344] Trial 80 finished with value: 0.13088954384718712 and parameters: {'colsample_bylevel': 0.06280087822832632, 'learning_rate': 0.08666988536050456, 'depth': 4, 'l2_leaf_reg': 4.29468157354246, 'subsample': 0.18212292966145463, 'bagging_temperature': 0.9370546569779352, 'model_size_reg': 0.12383225074863854, 'boosting_type': 'Plain'}. Best is trial 74 with value: 0.12640867299816086.
[I 2023-06-16 20:38:40,332] Trial 81 finished with value: 0.1290648375658185 and parameters: {'colsample_bylevel': 0.05334071039513488, 'learning_rate': 0.0720410660004371, 

...

[I 2023-06-16 21:15:53,085] Trial 939 finished with value: 0.12838022403323626 and parameters: {'colsample_bylevel': 0.0870907220081733, 'learning_rate': 0.07074873612914988, 'depth': 2, 'l2_leaf_reg': 19.149020590541554, 'subsample': 0.8398498955364825, 'bagging_temperature': 6.827338630041765, 'model_size_reg': 0.134185078605815, 'boosting_type': 'Plain'}. Best is trial 318 with value: 0.12432783178140529.
[I 2023-06-16 21:15:55,535] Trial 940 finished with value: 0.1336670616849474 and parameters: {'colsample_bylevel': 0.07335208279670011, 'learning_rate': 0.022361766516712754, 'depth': 4, 'l2_leaf_reg': 15.423796993216383, 'subsample': 0.46030331956315224, 'bagging_temperature': 3.8821726309754023, 'model_size_reg': 0.7901732708563248, 'boosting_type': 'Plain'}. Best is trial 318 with value: 0.12432783178140529.
[I 2023-06-16 21:15:57,306] Trial 941 finished with value: 0.13160090198234936 and parameters: {'colsample_bylevel': 0.08110810431396694, 'learning_rate': 0.094856885050626

[I 2023-06-16 21:16:33,015] Trial 959 finished with value: 0.12665727038654032 and parameters: {'colsample_bylevel': 0.08574466761806508, 'learning_rate': 0.09785819226551254, 'depth': 4, 'l2_leaf_reg': 11.950869208411438, 'subsample': 0.747963591938224, 'bagging_temperature': 3.030471334150405, 'model_size_reg': 0.23380243362928113, 'boosting_type': 'Plain'}. Best is trial 318 with value: 0.12432783178140529.
[I 2023-06-16 21:16:34,886] Trial 960 finished with value: 0.1328113616897245 and parameters: {'colsample_bylevel': 0.04154247623851098, 'learning_rate': 0.09017424274736015, 'depth': 3, 'l2_leaf_reg': 14.545388278240365, 'subsample': 0.20149661916725387, 'bagging_temperature': 6.285129973311095, 'model_size_reg': 0.45047011260034237, 'boosting_type': 'Plain'}. Best is trial 318 with value: 0.12432783178140529.
[I 2023-06-16 21:16:36,646] Trial 961 finished with value: 0.1332990794024464 and parameters: {'colsample_bylevel': 0.029196199247476753, 'learning_rate': 0.08566669870933

[I 2023-06-16 21:17:19,872] Trial 979 finished with value: 0.12863846317206853 and parameters: {'colsample_bylevel': 0.05364424254260714, 'learning_rate': 0.08234701002369599, 'depth': 2, 'l2_leaf_reg': 10.169475039603023, 'subsample': 0.9365589422522631, 'bagging_temperature': 2.425559451049911, 'model_size_reg': 0.10430196119311504, 'boosting_type': 'Plain'}. Best is trial 318 with value: 0.12432783178140529.
[I 2023-06-16 21:17:21,288] Trial 980 finished with value: 0.13131160137498152 and parameters: {'colsample_bylevel': 0.012375994137582773, 'learning_rate': 0.09974506167851292, 'depth': 4, 'l2_leaf_reg': 0.589886705135066, 'subsample': 0.35052209278905544, 'bagging_temperature': 6.431824886024593, 'model_size_reg': 0.20111878324537108, 'boosting_type': 'Plain'}. Best is trial 318 with value: 0.12432783178140529.
[I 2023-06-16 21:17:23,210] Trial 981 finished with value: 0.14337509061453563 and parameters: {'colsample_bylevel': 0.062351426370909686, 'learning_rate': 0.01070825698

[I 2023-06-16 21:18:17,139] Trial 999 finished with value: 0.16796137478288006 and parameters: {'colsample_bylevel': 0.0881672088152741, 'learning_rate': 0.005166622944458181, 'depth': 3, 'l2_leaf_reg': 15.212863229588068, 'subsample': 0.18692868363996085, 'bagging_temperature': 2.246618307536189, 'model_size_reg': 0.5178583340746605, 'boosting_type': 'Plain'}. Best is trial 318 with value: 0.12432783178140529.


Number of finished trials: 1000
Best trial: {'colsample_bylevel': 0.08309602563537534, 'learning_rate': 0.08286145675756133, 'depth': 4, 'l2_leaf_reg': 14.555249413444315, 'subsample': 0.9097411584295835, 'bagging_temperature': 3.177590955252409, 'model_size_reg': 0.3808343022980778, 'boosting_type': 'Plain'}


In [ ]:
# The shuffle function seems to create an error message in the catboost study, so it's being placed after that study. 

In [26]:
X,y = shuffle(X,y, random_state=42)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [121]:
def xgb_objective(trial):
    
    param = {  
        #'tree_method':method,
        #'objective': 'reg:squaredlogerror',
        'eval_metric': 'rmse',
        'lambda': trial.suggest_float('lambda', 0, 10.0),
        'alpha': trial.suggest_float('alpha', 0, 10.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1,1.0),
        'subsample': trial.suggest_float('subsample', 0.2,1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001,0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100,3000),
        'max_depth': trial.suggest_categorical('max_depth', [2,3,4,5,6,7,8,9,10]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'num_parallel_tree': trial.suggest_int('num_parallel_tree',1,1),
        }

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    
    
    return rmse

In [123]:
study_xgb = optuna.create_study(sampler=TPESampler(), direction='minimize')
study_xgb.optimize(xgb_objective, n_trials=1000)
print('Number of finished trials:', len(study_xgb.trials))
print('Best trial:', study_xgb.best_trial.params)

[I 2023-06-15 04:11:09,686] A new study created in memory with name: no-name-b3e44794-8152-48d0-b603-b800647a8cca
[I 2023-06-15 04:11:13,338] Trial 0 finished with value: 0.1498515336877748 and parameters: {'lambda': 6.162689645278529, 'alpha': 9.812666940780812, 'colsample_bytree': 0.47645024522659707, 'subsample': 0.6652532753971889, 'learning_rate': 0.02076914171202428, 'n_estimators': 1627, 'max_depth': 4, 'min_child_weight': 4, 'num_parallel_tree': 1}. Best is trial 0 with value: 0.1498515336877748.
[I 2023-06-15 04:11:15,095] Trial 1 finished with value: 0.13553165285224672 and parameters: {'lambda': 2.4485717078667477, 'alpha': 3.858171105025998, 'colsample_bytree': 0.46473113414127765, 'subsample': 0.3560299602307323, 'learning_rate': 0.052654261454556935, 'n_estimators': 436, 'max_depth': 10, 'min_child_weight': 4, 'num_parallel_tree': 1}. Best is trial 1 with value: 0.13553165285224672.
[I 2023-06-15 04:11:23,703] Trial 2 finished with value: 0.1339710781929653 and parameters

[I 2023-06-15 04:12:58,052] Trial 20 finished with value: 0.11375049237369299 and parameters: {'lambda': 4.69394803510848, 'alpha': 3.0741794824595754, 'colsample_bytree': 0.10780281299632348, 'subsample': 0.8085303388224501, 'learning_rate': 0.06643699052696476, 'n_estimators': 2505, 'max_depth': 8, 'min_child_weight': 7, 'num_parallel_tree': 1}. Best is trial 18 with value: 0.11254284311403855.
[I 2023-06-15 04:13:06,081] Trial 21 finished with value: 0.11345616233799391 and parameters: {'lambda': 4.791450157909963, 'alpha': 2.9861031393437028, 'colsample_bytree': 0.12135711506102975, 'subsample': 0.8187753391150732, 'learning_rate': 0.06232141786057986, 'n_estimators': 2492, 'max_depth': 8, 'min_child_weight': 7, 'num_parallel_tree': 1}. Best is trial 18 with value: 0.11254284311403855.
[I 2023-06-15 04:13:12,825] Trial 22 finished with value: 0.12796710643665288 and parameters: {'lambda': 5.382999046612794, 'alpha': 4.683298203890884, 'colsample_bytree': 0.23487752871175965, 'subsa

[I 2023-06-15 04:15:11,375] Trial 40 finished with value: 0.12187419545829786 and parameters: {'lambda': 0.9426128640317175, 'alpha': 4.073025839277824, 'colsample_bytree': 0.1469504259596297, 'subsample': 0.7048302697046215, 'learning_rate': 0.07021020544150608, 'n_estimators': 1398, 'max_depth': 7, 'min_child_weight': 9, 'num_parallel_tree': 1}. Best is trial 18 with value: 0.11254284311403855.
[I 2023-06-15 04:15:19,047] Trial 41 finished with value: 0.1149172809735578 and parameters: {'lambda': 4.475785038794795, 'alpha': 3.4063654126994196, 'colsample_bytree': 0.10166764541712832, 'subsample': 0.8150072974376544, 'learning_rate': 0.06623305836565146, 'n_estimators': 2460, 'max_depth': 8, 'min_child_weight': 7, 'num_parallel_tree': 1}. Best is trial 18 with value: 0.11254284311403855.
[I 2023-06-15 04:15:28,072] Trial 42 finished with value: 0.11181480272746401 and parameters: {'lambda': 4.629479717731837, 'alpha': 2.8804171611667044, 'colsample_bytree': 0.10929298652199507, 'subsa

[I 2023-06-15 04:17:33,797] Trial 60 finished with value: 0.11830468516231453 and parameters: {'lambda': 3.7187547127356084, 'alpha': 1.0341202187679148, 'colsample_bytree': 0.29232405709343295, 'subsample': 0.8526640855748221, 'learning_rate': 0.05953156471274138, 'n_estimators': 1708, 'max_depth': 3, 'min_child_weight': 6, 'num_parallel_tree': 1}. Best is trial 42 with value: 0.11181480272746401.
[I 2023-06-15 04:17:37,567] Trial 61 finished with value: 0.11370522275736966 and parameters: {'lambda': 2.9311256146570077, 'alpha': 1.9134503313145472, 'colsample_bytree': 0.13203508711885428, 'subsample': 0.8945601062435289, 'learning_rate': 0.08401681129067055, 'n_estimators': 1157, 'max_depth': 3, 'min_child_weight': 7, 'num_parallel_tree': 1}. Best is trial 42 with value: 0.11181480272746401.
[I 2023-06-15 04:17:42,040] Trial 62 finished with value: 0.11585389409453654 and parameters: {'lambda': 2.712918725119128, 'alpha': 1.698970544980144, 'colsample_bytree': 0.19154363547369596, 'su

[I 2023-06-15 04:19:08,844] Trial 80 finished with value: 0.11893033367462783 and parameters: {'lambda': 4.569686559959336, 'alpha': 1.2159717897215097, 'colsample_bytree': 0.17133321052386116, 'subsample': 0.832855980026397, 'learning_rate': 0.06610547593805784, 'n_estimators': 2165, 'max_depth': 8, 'min_child_weight': 5, 'num_parallel_tree': 1}. Best is trial 77 with value: 0.1112519140689954.
[I 2023-06-15 04:19:16,168] Trial 81 finished with value: 0.11290844922892884 and parameters: {'lambda': 4.869508761869948, 'alpha': 1.7628504759753267, 'colsample_bytree': 0.14586222922814865, 'subsample': 0.8696253625727823, 'learning_rate': 0.07559595982007992, 'n_estimators': 2257, 'max_depth': 8, 'min_child_weight': 7, 'num_parallel_tree': 1}. Best is trial 77 with value: 0.1112519140689954.
[I 2023-06-15 04:19:24,381] Trial 82 finished with value: 0.11318349297328059 and parameters: {'lambda': 4.860742623991869, 'alpha': 1.6415712166253864, 'colsample_bytree': 0.13979954995921898, 'subsam

...

[I 2023-06-15 06:07:15,588] Trial 920 finished with value: 0.11954524565044834 and parameters: {'lambda': 3.8845246255904793, 'alpha': 0.17457436619588595, 'colsample_bytree': 0.5237156193491632, 'subsample': 0.506578969732619, 'learning_rate': 0.059307058523197786, 'n_estimators': 2240, 'max_depth': 4, 'min_child_weight': 2, 'num_parallel_tree': 1}. Best is trial 244 with value: 0.1062486335599419.
[I 2023-06-15 06:07:22,479] Trial 921 finished with value: 0.13046618423432918 and parameters: {'lambda': 6.277899240485497, 'alpha': 4.7627353309935945, 'colsample_bytree': 0.18539223336926816, 'subsample': 0.9309380592185459, 'learning_rate': 0.04623977477884583, 'n_estimators': 2139, 'max_depth': 9, 'min_child_weight': 2, 'num_parallel_tree': 1}. Best is trial 244 with value: 0.1062486335599419.
[I 2023-06-15 06:07:30,007] Trial 922 finished with value: 0.1156221520591984 and parameters: {'lambda': 0.5776749763361657, 'alpha': 2.586269875805674, 'colsample_bytree': 0.11644955599427549, '

[I 2023-06-15 06:09:46,222] Trial 940 finished with value: 0.12004251319233769 and parameters: {'lambda': 5.580012568021568, 'alpha': 3.064941367017858, 'colsample_bytree': 0.41296603657816755, 'subsample': 0.9168821778553673, 'learning_rate': 0.07312352327911567, 'n_estimators': 2230, 'max_depth': 3, 'min_child_weight': 2, 'num_parallel_tree': 1}. Best is trial 244 with value: 0.1062486335599419.
[I 2023-06-15 06:09:54,390] Trial 941 finished with value: 0.543242605272931 and parameters: {'lambda': 7.152968732993122, 'alpha': 5.906619298751497, 'colsample_bytree': 0.7711816516852998, 'subsample': 0.4216487529492091, 'learning_rate': 0.0014077826456668843, 'n_estimators': 2360, 'max_depth': 3, 'min_child_weight': 2, 'num_parallel_tree': 1}. Best is trial 244 with value: 0.1062486335599419.
[I 2023-06-15 06:10:02,468] Trial 942 finished with value: 0.10855040575281528 and parameters: {'lambda': 9.020054894528968, 'alpha': 0.31357795168190017, 'colsample_bytree': 0.1167358704536358, 'sub

[I 2023-06-15 06:12:18,916] Trial 960 finished with value: 0.12065247168080553 and parameters: {'lambda': 3.783646373511197, 'alpha': 0.43308992270684044, 'colsample_bytree': 0.9724500317604623, 'subsample': 0.37856918019679997, 'learning_rate': 0.052994021310455934, 'n_estimators': 2415, 'max_depth': 3, 'min_child_weight': 4, 'num_parallel_tree': 1}. Best is trial 244 with value: 0.1062486335599419.
[I 2023-06-15 06:12:26,196] Trial 961 finished with value: 0.1452525653003379 and parameters: {'lambda': 7.487801425670158, 'alpha': 7.924362372810515, 'colsample_bytree': 0.10020400336705927, 'subsample': 0.4458649945656099, 'learning_rate': 0.04778911933337036, 'n_estimators': 2316, 'max_depth': 2, 'min_child_weight': 1, 'num_parallel_tree': 1}. Best is trial 244 with value: 0.1062486335599419.
[I 2023-06-15 06:12:33,791] Trial 962 finished with value: 0.11019195508068852 and parameters: {'lambda': 5.5245259504903235, 'alpha': 1.3758078216833391, 'colsample_bytree': 0.13360133250327844, 

[I 2023-06-15 06:15:01,952] Trial 980 finished with value: 0.1393710716780284 and parameters: {'lambda': 2.6191924933182245, 'alpha': 9.578468920545674, 'colsample_bytree': 0.16984757683440732, 'subsample': 0.9628746499757667, 'learning_rate': 0.0350966078524204, 'n_estimators': 2215, 'max_depth': 10, 'min_child_weight': 2, 'num_parallel_tree': 1}. Best is trial 244 with value: 0.1062486335599419.
[I 2023-06-15 06:15:09,995] Trial 981 finished with value: 0.11741285009504017 and parameters: {'lambda': 1.2252159219805234, 'alpha': 1.6543899640217965, 'colsample_bytree': 0.33049668774063157, 'subsample': 0.9446145812160079, 'learning_rate': 0.049790067732401404, 'n_estimators': 2266, 'max_depth': 8, 'min_child_weight': 2, 'num_parallel_tree': 1}. Best is trial 244 with value: 0.1062486335599419.
[I 2023-06-15 06:15:19,274] Trial 982 finished with value: 0.1258406400597271 and parameters: {'lambda': 8.436907611620498, 'alpha': 0.6834152234994193, 'colsample_bytree': 0.8853606214197638, 's

Number of finished trials: 1000
Best trial: {'lambda': 5.096430599223015, 'alpha': 1.3223698329753615, 'colsample_bytree': 0.10366646849522436, 'subsample': 0.9705075073357825, 'learning_rate': 0.04681670801808327, 'n_estimators': 2340, 'max_depth': 3, 'min_child_weight': 2, 'num_parallel_tree': 1}


In [27]:
def bayesian_ridge_objective(trial):
    param = {  
        
        "alpha_init": trial.suggest_float ("alpha_init", 1.0, 4.0),
        "lambda_init": trial.suggest_float ("lambda_init", 1e-9, 1e-1)
        }        
    
    model = BayesianRidge (**param)
    model.fit(X,y)
    predictions = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, predictions, squared=False))
    
    return rmse

In [28]:
study_bayesian_ridge = optuna.create_study(sampler=TPESampler(), direction='minimize')
study_bayesian_ridge.optimize(bayesian_ridge_objective, n_trials=1000)
print('Number of finished trials:', len(study_bayesian_ridge.trials))
print('Best trial:', study_bayesian_ridge.best_trial.params)

[I 2023-06-15 15:32:07,211] A new study created in memory with name: no-name-7931f915-6bc6-4cbc-9512-03e3cf185c8d
[I 2023-06-15 15:32:07,313] Trial 0 finished with value: 0.3547672935093919 and parameters: {'alpha_init': 2.480873102295276, 'lambda_init': 0.012772866985845083}. Best is trial 0 with value: 0.3547672935093919.
[I 2023-06-15 15:32:07,325] Trial 1 finished with value: 0.35476714899834294 and parameters: {'alpha_init': 2.7799264689076066, 'lambda_init': 0.011733626492015994}. Best is trial 1 with value: 0.35476714899834294.
[I 2023-06-15 15:32:07,340] Trial 2 finished with value: 0.3547678545833306 and parameters: {'alpha_init': 2.106252757252474, 'lambda_init': 0.02359702290846118}. Best is trial 1 with value: 0.35476714899834294.
[I 2023-06-15 15:32:07,357] Trial 3 finished with value: 0.3547682618903762 and parameters: {'alpha_init': 2.4756391739193795, 'lambda_init': 0.056052018934838084}. Best is trial 1 with value: 0.35476714899834294.
[I 2023-06-15 15:32:07,373] Trial

[I 2023-06-15 15:32:08,270] Trial 36 finished with value: 0.3547668312061387 and parameters: {'alpha_init': 1.112711027651518, 'lambda_init': 0.05440782221098471}. Best is trial 22 with value: 0.35476646179590654.
[I 2023-06-15 15:32:08,300] Trial 37 finished with value: 0.3547666931436879 and parameters: {'alpha_init': 1.3123952894055226, 'lambda_init': 0.050658058737051176}. Best is trial 22 with value: 0.35476646179590654.
[I 2023-06-15 15:32:08,334] Trial 38 finished with value: 0.3547680065547064 and parameters: {'alpha_init': 1.8665818212141572, 'lambda_init': 0.026353722559054}. Best is trial 22 with value: 0.35476646179590654.
[I 2023-06-15 15:32:08,359] Trial 39 finished with value: 0.3547681416888014 and parameters: {'alpha_init': 2.08656325090761, 'lambda_init': 0.03712063703875145}. Best is trial 22 with value: 0.35476646179590654.
[I 2023-06-15 15:32:08,392] Trial 40 finished with value: 0.3547683205637317 and parameters: {'alpha_init': 1.588355024551446, 'lambda_init': 0.

[I 2023-06-15 15:32:09,259] Trial 72 finished with value: 0.3547683005607383 and parameters: {'alpha_init': 1.3657662203263916, 'lambda_init': 0.03376898950417789}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:09,282] Trial 73 finished with value: 0.3547666485142242 and parameters: {'alpha_init': 1.0680297678598014, 'lambda_init': 0.03844113839106654}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:09,305] Trial 74 finished with value: 0.3547666460287729 and parameters: {'alpha_init': 1.2927424206145832, 'lambda_init': 0.04635255955872551}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:09,329] Trial 75 finished with value: 0.3547666526757589 and parameters: {'alpha_init': 1.1822419308340146, 'lambda_init': 0.04282459934255856}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:09,353] Trial 76 finished with value: 0.35476815644344556 and parameters: {'alpha_init': 1.4693388188093661, 'lambda_init': 

[I 2023-06-15 15:32:10,125] Trial 108 finished with value: 0.35476659134463673 and parameters: {'alpha_init': 1.3032446889579279, 'lambda_init': 0.04308319573798738}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:10,146] Trial 109 finished with value: 0.3547682709105505 and parameters: {'alpha_init': 1.4927337409271255, 'lambda_init': 0.034480410021594105}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:10,172] Trial 110 finished with value: 0.3547682395345631 and parameters: {'alpha_init': 1.7271344401940125, 'lambda_init': 0.03726202297068182}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:10,204] Trial 111 finished with value: 0.354766506521596 and parameters: {'alpha_init': 1.7640280053696955, 'lambda_init': 0.05186163370219543}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:10,234] Trial 112 finished with value: 0.3547683310361888 and parameters: {'alpha_init': 1.812246452790874, 'lambda_ini

[I 2023-06-15 15:32:11,066] Trial 144 finished with value: 0.3547682522426475 and parameters: {'alpha_init': 1.912960467378071, 'lambda_init': 0.042409851253069814}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:11,090] Trial 145 finished with value: 0.3547665328647279 and parameters: {'alpha_init': 1.545614313314415, 'lambda_init': 0.047075161258734004}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:11,115] Trial 146 finished with value: 0.3547683065627838 and parameters: {'alpha_init': 1.800420375401094, 'lambda_init': 0.04515389176871697}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:11,144] Trial 147 finished with value: 0.35476831619103877 and parameters: {'alpha_init': 1.460117936825005, 'lambda_init': 0.037477379340492306}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:11,165] Trial 148 finished with value: 0.35476655129298745 and parameters: {'alpha_init': 1.6303271836529503, 'lambda_in

...

[I 2023-06-15 15:32:41,639] Trial 864 finished with value: 0.3547669899694098 and parameters: {'alpha_init': 1.3580848804028456, 'lambda_init': 0.09016320941793841}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:41,691] Trial 865 finished with value: 0.354766797648377 and parameters: {'alpha_init': 1.4782056712446072, 'lambda_init': 0.06805396708927969}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:41,758] Trial 866 finished with value: 0.3547664745074957 and parameters: {'alpha_init': 2.0200562028073366, 'lambda_init': 0.05696263292662686}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:41,808] Trial 867 finished with value: 0.35476823676145186 and parameters: {'alpha_init': 2.3939482550476625, 'lambda_init': 0.05134618825361402}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:41,859] Trial 868 finished with value: 0.3547679581413057 and parameters: {'alpha_init': 1.8099364516704612, 'lambda_ini

[I 2023-06-15 15:32:43,692] Trial 900 finished with value: 0.35476833072516 and parameters: {'alpha_init': 1.4106087566879606, 'lambda_init': 0.037525765633287814}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:43,749] Trial 901 finished with value: 0.3547683391793745 and parameters: {'alpha_init': 1.38385346092541, 'lambda_init': 0.03760608861501092}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:43,808] Trial 902 finished with value: 0.35476831060440744 and parameters: {'alpha_init': 1.451350869078861, 'lambda_init': 0.03675312049204965}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:43,856] Trial 903 finished with value: 0.3547665245163777 and parameters: {'alpha_init': 1.302890914672539, 'lambda_init': 0.039236448442161696}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:43,901] Trial 904 finished with value: 0.3547665162941727 and parameters: {'alpha_init': 1.3649874767681764, 'lambda_init':

[I 2023-06-15 15:32:45,818] Trial 936 finished with value: 0.3547682560458323 and parameters: {'alpha_init': 1.577717422543467, 'lambda_init': 0.0352676422397429}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:45,875] Trial 937 finished with value: 0.35476669727607546 and parameters: {'alpha_init': 1.1360263823420085, 'lambda_init': 0.044142226742603326}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:45,926] Trial 938 finished with value: 0.35476656599150164 and parameters: {'alpha_init': 2.3615110100394965, 'lambda_init': 0.07529715806189724}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:46,173] Trial 939 finished with value: 0.35476798248014624 and parameters: {'alpha_init': 2.7940207757020294, 'lambda_init': 0.03796902787178774}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:46,234] Trial 940 finished with value: 0.3547669778649545 and parameters: {'alpha_init': 1.4560600523792304, 'lambda_i

[I 2023-06-15 15:32:48,252] Trial 972 finished with value: 0.35476660531526977 and parameters: {'alpha_init': 1.185548156801582, 'lambda_init': 0.03999686683044354}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:48,316] Trial 973 finished with value: 0.354768324213074 and parameters: {'alpha_init': 1.6137564874665697, 'lambda_init': 0.04224189328925162}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:48,367] Trial 974 finished with value: 0.3547679189078809 and parameters: {'alpha_init': 1.5139162052089306, 'lambda_init': 0.018656517601865298}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:48,421] Trial 975 finished with value: 0.35476678591827027 and parameters: {'alpha_init': 1.894521347454272, 'lambda_init': 0.08543808140767278}. Best is trial 58 with value: 0.3547664519999626.
[I 2023-06-15 15:32:48,481] Trial 976 finished with value: 0.35476698629436243 and parameters: {'alpha_init': 1.6834147592226298, 'lambda_in

Number of finished trials: 1000
Best trial: {'alpha_init': 1.4228919418021304, 'lambda_init': 0.03899454131279402}


In [31]:
def lightgbm_objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "n_estimators": 10000,
        "verbosity": -1,
        "bagging_freq": 4,
        "bagging_seed": 8,
        "feature_fraction": trial.suggest_float("feature_fraction", 0.1, 0.9),
        "learning_rate": trial.suggest_float("learning_rate", .001, 0.1, log=True),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.1, 0.9),
        "num_leaves": trial.suggest_int("num_leaves", 2, 76),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "min_sum_hessian_in_leaf": trial.suggest_int("min_sum_hessian_in_leaf", 1, 100),
    }
    
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    model = LGBMRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, squared=False))
    
    return rmse

In [32]:
study_lightgbm = optuna.create_study(sampler=TPESampler(), direction='minimize')
study_lightgbm.optimize(lightgbm_objective, n_trials=1000)
print('Number of finished trials:', len(study_lightgbm.trials))
print('Best trial:', study_lightgbm.best_trial.params)

[I 2023-06-15 15:47:29,830] A new study created in memory with name: no-name-2036eb3e-5205-49a2-b491-f1326dd11023
[I 2023-06-15 15:47:35,674] Trial 0 finished with value: 0.35793777037460484 and parameters: {'feature_fraction': 0.8648702261918263, 'learning_rate': 0.010388166997318723, 'bagging_fraction': 0.5458485841175765, 'num_leaves': 56, 'subsample': 0.5785842764301418, 'min_sum_hessian_in_leaf': 61}. Best is trial 0 with value: 0.35793777037460484.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.5512857679801416, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5512857679801416
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=60, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=60
[LightGBM] [Warning] bagging_fraction is set=0.574038569325351, subsample=0.10077454852498961 will be ignored. Current value: bagging_fraction=0.574038569325351
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


[I 2023-06-15 15:47:41,467] Trial 1 finished with value: 0.3618421486904888 and parameters: {'feature_fraction': 0.5512857679801416, 'learning_rate': 0.023557590507560202, 'bagging_fraction': 0.574038569325351, 'num_leaves': 40, 'subsample': 0.10077454852498961, 'min_sum_hessian_in_leaf': 60}. Best is trial 0 with value: 0.35793777037460484.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.5418961599984453, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5418961599984453
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=92, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=92
[LightGBM] [Warning] bagging_fraction is set=0.2705934623483922, subsample=0.9326751001651747 will be ignored. Current value: bagging_fraction=0.2705934623483922
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


[I 2023-06-15 15:47:44,633] Trial 2 finished with value: 0.38750854486626857 and parameters: {'feature_fraction': 0.5418961599984453, 'learning_rate': 0.0010844456011230104, 'bagging_fraction': 0.2705934623483922, 'num_leaves': 4, 'subsample': 0.9326751001651747, 'min_sum_hessian_in_leaf': 92}. Best is trial 0 with value: 0.35793777037460484.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


...

[LightGBM] [Warning] feature_fraction is set=0.2836775621660017, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2836775621660017
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=4, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=4
[LightGBM] [Warning] bagging_fraction is set=0.4069664839105537, subsample=0.9634965698136012 will be ignored. Current value: bagging_fraction=0.4069664839105537
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


[I 2023-06-15 18:25:52,427] Trial 998 finished with value: 0.3353615896522654 and parameters: {'feature_fraction': 0.2836775621660017, 'learning_rate': 0.00301798735781624, 'bagging_fraction': 0.4069664839105537, 'num_leaves': 6, 'subsample': 0.9634965698136012, 'min_sum_hessian_in_leaf': 4}. Best is trial 194 with value: 0.3305620323977541.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.2052980368769869, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2052980368769869
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=10, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.4346875850867194, subsample=0.2722534328050548 will be ignored. Current value: bagging_fraction=0.4346875850867194
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


[I 2023-06-15 18:26:01,398] Trial 999 finished with value: 0.353993945670483 and parameters: {'feature_fraction': 0.2052980368769869, 'learning_rate': 0.0939554510474155, 'bagging_fraction': 0.4346875850867194, 'num_leaves': 10, 'subsample': 0.2722534328050548, 'min_sum_hessian_in_leaf': 10}. Best is trial 194 with value: 0.3305620323977541.


Number of finished trials: 1000
Best trial: {'feature_fraction': 0.1309662269268009, 'learning_rate': 0.0019200325083996161, 'bagging_fraction': 0.4240987934393466, 'num_leaves': 9, 'subsample': 0.40495582495561083, 'min_sum_hessian_in_leaf': 1}


In [119]:
def svr_objective(trial):
    params = {
        "C": trial.suggest_float("C", 1.0, 30.0),
        "epsilon": trial.suggest_float("epsilon", .001, 0.2),
        "gamma": trial.suggest_float("gamma", .0001, .001)
        
    }

    model = make_pipeline(RobustScaler(), SVR(**params))
    model.fit(X, y)
    predictions = model.predict(X)
    rmse = mean_squared_error(y, predictions, squared=False)
    
    return rmse


In [120]:
study_svr = optuna.create_study(sampler=TPESampler(), direction='minimize')
study_svr.optimize(svr_objective, n_trials=1000)
print('Number of finished trials:', len(study_svr.trials))
print('Best trial:', study_svr.best_trial.params)

[I 2023-06-15 04:00:49,979] A new study created in memory with name: no-name-7d2b9340-8973-45a4-be7f-42cb3be96b93
[I 2023-06-15 04:00:50,220] Trial 0 finished with value: 0.12000923517776027 and parameters: {'C': 13.414609389142036, 'epsilon': 0.09682592170557598, 'gamma': 0.0003612302416176735}. Best is trial 0 with value: 0.12000923517776027.
[I 2023-06-15 04:00:50,420] Trial 1 finished with value: 0.1113628972918608 and parameters: {'C': 22.53091195582332, 'epsilon': 0.11357673939914321, 'gamma': 0.000637549968713913}. Best is trial 1 with value: 0.1113628972918608.
[I 2023-06-15 04:00:50,911] Trial 2 finished with value: 0.09354376491966578 and parameters: {'C': 22.349542195942803, 'epsilon': 0.036471654020282004, 'gamma': 0.0009760672004762042}. Best is trial 2 with value: 0.09354376491966578.
[I 2023-06-15 04:00:51,391] Trial 3 finished with value: 0.10414473738460069 and parameters: {'C': 27.50031454454288, 'epsilon': 0.03396988365369881, 'gamma': 0.0006006685388117762}. Best is

[I 2023-06-15 04:01:06,863] Trial 33 finished with value: 0.09267501580260683 and parameters: {'C': 27.99332244630357, 'epsilon': 0.028480384303830656, 'gamma': 0.0008880904251347339}. Best is trial 15 with value: 0.08856257149693546.
[I 2023-06-15 04:01:07,612] Trial 34 finished with value: 0.09002245062139509 and parameters: {'C': 25.99840224589608, 'epsilon': 0.010736057419461213, 'gamma': 0.0009977598773832684}. Best is trial 15 with value: 0.08856257149693546.
[I 2023-06-15 04:01:08,073] Trial 35 finished with value: 0.0944277885020209 and parameters: {'C': 23.44059355453964, 'epsilon': 0.03930235251573487, 'gamma': 0.0009335465358758359}. Best is trial 15 with value: 0.08856257149693546.
[I 2023-06-15 04:01:08,962] Trial 36 finished with value: 0.09082325333023485 and parameters: {'C': 28.13545587123576, 'epsilon': 0.0013579361634492, 'gamma': 0.0009517359406274718}. Best is trial 15 with value: 0.08856257149693546.
[I 2023-06-15 04:01:09,449] Trial 37 finished with value: 0.1086

[I 2023-06-15 04:01:30,409] Trial 67 finished with value: 0.0923682932648551 and parameters: {'C': 27.93109567995214, 'epsilon': 0.03362613282853073, 'gamma': 0.0009034877680777707}. Best is trial 15 with value: 0.08856257149693546.
[I 2023-06-15 04:01:30,945] Trial 68 finished with value: 0.09556184000077615 and parameters: {'C': 26.25460875934038, 'epsilon': 0.041422837916700525, 'gamma': 0.0008561222530455504}. Best is trial 15 with value: 0.08856257149693546.
[I 2023-06-15 04:01:31,532] Trial 69 finished with value: 0.08897064626198972 and parameters: {'C': 29.03247398111965, 'epsilon': 0.0326198709854004, 'gamma': 0.0009838740174915352}. Best is trial 15 with value: 0.08856257149693546.
[I 2023-06-15 04:01:32,153] Trial 70 finished with value: 0.08926774820299503 and parameters: {'C': 27.711410272590435, 'epsilon': 0.033817241027512636, 'gamma': 0.0009996037260003115}. Best is trial 15 with value: 0.08856257149693546.
[I 2023-06-15 04:01:32,794] Trial 71 finished with value: 0.088

[I 2023-06-15 04:01:58,920] Trial 101 finished with value: 0.08889117560091565 and parameters: {'C': 29.993863407239143, 'epsilon': 0.0010908859019229163, 'gamma': 0.000977553625906693}. Best is trial 81 with value: 0.08804460578737672.
[I 2023-06-15 04:02:01,147] Trial 102 finished with value: 0.08796186009905049 and parameters: {'C': 29.232263191007416, 'epsilon': 0.02428032478572906, 'gamma': 0.0009991591285969945}. Best is trial 102 with value: 0.08796186009905049.
[I 2023-06-15 04:02:03,124] Trial 103 finished with value: 0.089326049742513 and parameters: {'C': 29.44971564066154, 'epsilon': 0.00875857309888601, 'gamma': 0.0009602866297847163}. Best is trial 102 with value: 0.08796186009905049.
[I 2023-06-15 04:02:04,614] Trial 104 finished with value: 0.08903887722856012 and parameters: {'C': 28.78119577743265, 'epsilon': 0.004407352993896336, 'gamma': 0.0009854523104336494}. Best is trial 102 with value: 0.08796186009905049.
[I 2023-06-15 04:02:06,228] Trial 105 finished with val

[I 2023-06-15 04:02:46,396] Trial 134 finished with value: 0.08828695112431392 and parameters: {'C': 28.560281178180297, 'epsilon': 0.022620550514032174, 'gamma': 0.0009995570503506495}. Best is trial 124 with value: 0.08786694412290184.
[I 2023-06-15 04:02:47,236] Trial 135 finished with value: 0.08878834544413985 and parameters: {'C': 28.545256741464033, 'epsilon': 0.02430510752040601, 'gamma': 0.000986604083436892}. Best is trial 124 with value: 0.08786694412290184.
[I 2023-06-15 04:02:48,070] Trial 136 finished with value: 0.08904040261160341 and parameters: {'C': 27.30586217702309, 'epsilon': 0.018573453590073712, 'gamma': 0.0009983399409901547}. Best is trial 124 with value: 0.08786694412290184.
[I 2023-06-15 04:02:48,592] Trial 137 finished with value: 0.10247931632042884 and parameters: {'C': 26.32591118022577, 'epsilon': 0.0373171096234704, 'gamma': 0.0006603876564420547}. Best is trial 124 with value: 0.08786694412290184.
[I 2023-06-15 04:02:49,261] Trial 138 finished with va

...

[I 2023-06-15 04:09:56,864] Trial 893 finished with value: 0.11389851885755088 and parameters: {'C': 7.476155322653055, 'epsilon': 0.11940193780889187, 'gamma': 0.0009987552969571167}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:09:57,026] Trial 894 finished with value: 0.11187065361197798 and parameters: {'C': 29.994982224403458, 'epsilon': 0.1679482537608038, 'gamma': 0.0009746921016910685}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:09:57,710] Trial 895 finished with value: 0.09918519920389444 and parameters: {'C': 28.347534881255115, 'epsilon': 0.01589407548046008, 'gamma': 0.0007117460720487936}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:09:58,301] Trial 896 finished with value: 0.09433577151350639 and parameters: {'C': 19.903772075363648, 'epsilon': 0.02383075075572213, 'gamma': 0.0009995540747818998}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:09:58,493] Trial 897 finished with va

[I 2023-06-15 04:10:11,115] Trial 926 finished with value: 0.11884918471848918 and parameters: {'C': 2.3258265819128034, 'epsilon': 0.03130641605425876, 'gamma': 0.000982149257745791}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:10:11,457] Trial 927 finished with value: 0.09696157048481054 and parameters: {'C': 28.404013047856775, 'epsilon': 0.07553408674350007, 'gamma': 0.000886638346391608}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:10:12,069] Trial 928 finished with value: 0.09490234962153378 and parameters: {'C': 29.991842035497605, 'epsilon': 0.024955629531948815, 'gamma': 0.0007989251437191402}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:10:12,236] Trial 929 finished with value: 0.1098768260405304 and parameters: {'C': 23.5737220244516, 'epsilon': 0.1492949472061194, 'gamma': 0.0009367409415428644}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:10:12,567] Trial 930 finished with value

[I 2023-06-15 04:10:25,443] Trial 959 finished with value: 0.10980173503572489 and parameters: {'C': 29.994320939216397, 'epsilon': 0.15819632428654457, 'gamma': 0.0009400526027294547}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:10:25,592] Trial 960 finished with value: 0.12807754944881589 and parameters: {'C': 16.296816519165233, 'epsilon': 0.1774957319239271, 'gamma': 0.0002967823940967048}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:10:26,128] Trial 961 finished with value: 0.10055376083068995 and parameters: {'C': 13.949334960225338, 'epsilon': 0.03444357758691554, 'gamma': 0.0009696669723048663}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:10:26,336] Trial 962 finished with value: 0.11014049058441581 and parameters: {'C': 28.990792915359357, 'epsilon': 0.16109155092187102, 'gamma': 0.0009859871091416259}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:10:26,855] Trial 963 finished with v

[I 2023-06-15 04:10:43,512] Trial 992 finished with value: 0.10733336239064907 and parameters: {'C': 7.8868024086564565, 'epsilon': 0.03187793016217193, 'gamma': 0.0009770103140900389}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:10:44,108] Trial 993 finished with value: 0.09017936760330422 and parameters: {'C': 28.577852017919717, 'epsilon': 0.038045383370942785, 'gamma': 0.0009655528585881552}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:10:44,405] Trial 994 finished with value: 0.12096265066088814 and parameters: {'C': 14.06945528826228, 'epsilon': 0.08022533734733171, 'gamma': 0.00028933292708611995}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:10:44,626] Trial 995 finished with value: 0.11086620062712907 and parameters: {'C': 27.380528391580253, 'epsilon': 0.12710803394668857, 'gamma': 0.0006522291405750167}. Best is trial 290 with value: 0.08742437135795915.
[I 2023-06-15 04:10:45,189] Trial 996 finished with

Number of finished trials: 1000
Best trial: {'C': 29.994348636600485, 'epsilon': 0.01948234199552718, 'gamma': 0.0009992835981754227}


In [79]:
def dtr_objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 1,12),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1,10),
        "min_weight_fraction_leaf": trial.suggest_int("min_weight_fraction_leaf", 0.1, 0.9),
        "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 10,90),
        "max_features": trial.suggest_categorical("max_features", ["auto","log2","sqrt",None]),
        
    }

    model = DecisionTreeRegressor(**params)
    model.fit(X, y)
    predictions = model.predict(X)
    rmse = mean_squared_error(y, predictions, squared=False)
    return rmse

In [80]:
study_dtr = optuna.create_study(sampler=TPESampler(), direction='minimize')
study_dtr.optimize(dtr_objective, n_trials=1000)
print('Number of finished trials:', len(study_dtr.trials))
print('Best trial:', study_dtr.best_trial.params)

[I 2023-06-15 02:14:49,694] A new study created in memory with name: no-name-22a6b780-866d-4d91-969c-3f66ad159bd1
[I 2023-06-15 02:14:49,716] Trial 0 finished with value: 0.29300932996394924 and parameters: {'max_depth': 1, 'min_samples_leaf': 9, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 52, 'max_features': None}. Best is trial 0 with value: 0.29300932996394924.
[I 2023-06-15 02:14:49,739] Trial 1 finished with value: 0.15261995176587217 and parameters: {'max_depth': 12, 'min_samples_leaf': 10, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 32, 'max_features': None}. Best is trial 1 with value: 0.15261995176587217.
[I 2023-06-15 02:14:49,749] Trial 2 finished with value: 0.21712772679712222 and parameters: {'max_depth': 4, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 39, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.15261995176587217.
[I 2023-06-15 02:14:49,774] Trial 3 finished with value: 0.12207156552590474 and parameters: {'max_depth': 1

[I 2023-06-15 02:14:50,647] Trial 30 finished with value: 0.11521200664206703 and parameters: {'max_depth': 11, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 71, 'max_features': 'auto'}. Best is trial 26 with value: 0.11063719949588675.
[I 2023-06-15 02:14:50,680] Trial 31 finished with value: 0.11180226395645004 and parameters: {'max_depth': 8, 'min_samples_leaf': 4, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 81, 'max_features': 'auto'}. Best is trial 26 with value: 0.11063719949588675.
[I 2023-06-15 02:14:50,724] Trial 32 finished with value: 0.11152754146360844 and parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 81, 'max_features': 'auto'}. Best is trial 26 with value: 0.11063719949588675.
[I 2023-06-15 02:14:50,762] Trial 33 finished with value: 0.10964087125121645 and parameters: {'max_depth': 12, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 83, 'max_features': 'auto'}.

[I 2023-06-15 02:14:51,748] Trial 60 finished with value: 0.10644514807279308 and parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 87, 'max_features': None}. Best is trial 51 with value: 0.10435306282180538.
[I 2023-06-15 02:14:51,784] Trial 61 finished with value: 0.109803265560013 and parameters: {'max_depth': 7, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 90, 'max_features': None}. Best is trial 51 with value: 0.10435306282180538.
[I 2023-06-15 02:14:51,816] Trial 62 finished with value: 0.15766561706674306 and parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 90, 'max_features': None}. Best is trial 51 with value: 0.10435306282180538.
[I 2023-06-15 02:14:51,855] Trial 63 finished with value: 0.1112230224660354 and parameters: {'max_depth': 9, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 78, 'max_features': None}. Best is tria

[I 2023-06-15 02:14:52,790] Trial 90 finished with value: 0.13652274460361302 and parameters: {'max_depth': 11, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 88, 'max_features': 'sqrt'}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:14:52,830] Trial 91 finished with value: 0.10489405874906275 and parameters: {'max_depth': 11, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 86, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:14:52,869] Trial 92 finished with value: 0.10489405874906275 and parameters: {'max_depth': 11, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 86, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:14:52,907] Trial 93 finished with value: 0.10543363332448713 and parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 85, 'max_features': None}. Best

[I 2023-06-15 02:14:54,025] Trial 120 finished with value: 0.1070906731689731 and parameters: {'max_depth': 12, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 82, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:14:54,068] Trial 121 finished with value: 0.10381834416124357 and parameters: {'max_depth': 12, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 88, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:14:54,114] Trial 122 finished with value: 0.10543363332448713 and parameters: {'max_depth': 12, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 85, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:14:54,172] Trial 123 finished with value: 0.13714981374233112 and parameters: {'max_depth': 12, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 42, 'max_features': None}. Bes

...

[I 2023-06-15 02:15:56,513] Trial 900 finished with value: 0.10435306282180537 and parameters: {'max_depth': 12, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 87, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:15:56,638] Trial 901 finished with value: 0.10631267472880757 and parameters: {'max_depth': 10, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 90, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:15:56,745] Trial 902 finished with value: 0.12132272560814154 and parameters: {'max_depth': 12, 'min_samples_leaf': 9, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 85, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:15:56,846] Trial 903 finished with value: 0.15026804343162042 and parameters: {'max_depth': 11, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 88, 'max_features': 'log2'}. 

[I 2023-06-15 02:15:59,830] Trial 930 finished with value: 0.24802402151228897 and parameters: {'max_depth': 2, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 90, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:15:59,923] Trial 931 finished with value: 0.10381834416124355 and parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 88, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:16:00,022] Trial 932 finished with value: 0.1350651691844291 and parameters: {'max_depth': 12, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 44, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:16:00,111] Trial 933 finished with value: 0.13104747955174642 and parameters: {'max_depth': 11, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 86, 'max_features': 'sqrt'}. Be

[I 2023-06-15 02:16:03,192] Trial 960 finished with value: 0.1033031162079817 and parameters: {'max_depth': 12, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 89, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:16:03,291] Trial 961 finished with value: 0.10495463618616262 and parameters: {'max_depth': 11, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 90, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:16:03,415] Trial 962 finished with value: 0.10435306282180538 and parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 87, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:16:03,524] Trial 963 finished with value: 0.1033031162079817 and parameters: {'max_depth': 12, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 89, 'max_features': None}. Best

[I 2023-06-15 02:16:06,711] Trial 990 finished with value: 0.10489405874906275 and parameters: {'max_depth': 11, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 86, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:16:06,832] Trial 991 finished with value: 0.10381834416124355 and parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 88, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:16:06,929] Trial 992 finished with value: 0.1033031162079817 and parameters: {'max_depth': 12, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 89, 'max_features': None}. Best is trial 73 with value: 0.10279874517987195.
[I 2023-06-15 02:16:07,030] Trial 993 finished with value: 0.10279874517987196 and parameters: {'max_depth': 11, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 90, 'max_features': None}. Bes

Number of finished trials: 1000
Best trial: {'max_depth': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_leaf_nodes': 90, 'max_features': None}


In [34]:
def rfr_objective(trial):
    
    param = {  
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'max_depth': trial.suggest_int('max_depth', 1, 50),
            'min_samples_split': trial.suggest_int('min_samples_split', 5, 200, step=5),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 60),
            'verbose': 0
        }

    model = RandomForestRegressor(**param)
    model.fit(X, y)
    predictions = model.predict(X)
    rmse = mean_squared_error(y, predictions, squared=False)
    
    return rmse

In [35]:
study_rfr = optuna.create_study(sampler=TPESampler(), direction='minimize')
study_rfr.optimize(rfr_objective, n_trials=1000)
print('Number of finished trials:', len(study_rfr.trials))
print('Best trial:', study_rfr.best_trial.params)

[I 2023-06-15 10:36:28,393] A new study created in memory with name: no-name-7030efbf-0fba-4439-a432-40fb4b2387b8
[I 2023-06-15 10:36:31,882] Trial 0 finished with value: 0.17258575995310044 and parameters: {'n_estimators': 595, 'max_depth': 39, 'min_samples_split': 25, 'min_samples_leaf': 56}. Best is trial 0 with value: 0.17258575995310044.
[I 2023-06-15 10:36:33,560] Trial 1 finished with value: 0.18938764967926852 and parameters: {'n_estimators': 340, 'max_depth': 12, 'min_samples_split': 190, 'min_samples_leaf': 56}. Best is trial 0 with value: 0.17258575995310044.
[I 2023-06-15 10:36:34,857] Trial 2 finished with value: 0.17211858949617775 and parameters: {'n_estimators': 220, 'max_depth': 21, 'min_samples_split': 145, 'min_samples_leaf': 2}. Best is trial 2 with value: 0.17211858949617775.
[I 2023-06-15 10:36:39,585] Trial 3 finished with value: 0.1768021088379942 and parameters: {'n_estimators': 816, 'max_depth': 9, 'min_samples_split': 150, 'min_samples_leaf': 26}. Best is tri

[I 2023-06-15 10:38:43,485] Trial 33 finished with value: 0.13685475133848887 and parameters: {'n_estimators': 465, 'max_depth': 32, 'min_samples_split': 55, 'min_samples_leaf': 11}. Best is trial 10 with value: 0.05741849099912848.
[I 2023-06-15 10:38:45,017] Trial 34 finished with value: 0.18664548499851635 and parameters: {'n_estimators': 263, 'max_depth': 35, 'min_samples_split': 200, 'min_samples_leaf': 1}. Best is trial 10 with value: 0.05741849099912848.
[I 2023-06-15 10:38:50,174] Trial 35 finished with value: 0.0817607491362445 and parameters: {'n_estimators': 385, 'max_depth': 21, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 10 with value: 0.05741849099912848.
[I 2023-06-15 10:38:52,336] Trial 36 finished with value: 0.08912206847370861 and parameters: {'n_estimators': 170, 'max_depth': 14, 'min_samples_split': 5, 'min_samples_leaf': 5}. Best is trial 10 with value: 0.05741849099912848.
[I 2023-06-15 10:38:54,459] Trial 37 finished with value: 0.0902624490376

[I 2023-06-15 10:40:49,263] Trial 67 finished with value: 0.15878502829404373 and parameters: {'n_estimators': 394, 'max_depth': 29, 'min_samples_split': 25, 'min_samples_leaf': 40}. Best is trial 10 with value: 0.05741849099912848.
[I 2023-06-15 10:40:54,692] Trial 68 finished with value: 0.1377531898025217 and parameters: {'n_estimators': 632, 'max_depth': 40, 'min_samples_split': 45, 'min_samples_leaf': 20}. Best is trial 10 with value: 0.05741849099912848.
[I 2023-06-15 10:41:00,469] Trial 69 finished with value: 0.08559297885001224 and parameters: {'n_estimators': 444, 'max_depth': 17, 'min_samples_split': 10, 'min_samples_leaf': 4}. Best is trial 10 with value: 0.05741849099912848.
[I 2023-06-15 10:41:07,264] Trial 70 finished with value: 0.10638077717187923 and parameters: {'n_estimators': 586, 'max_depth': 31, 'min_samples_split': 20, 'min_samples_leaf': 6}. Best is trial 10 with value: 0.05741849099912848.
[I 2023-06-15 10:41:13,462] Trial 71 finished with value: 0.08141255571

[I 2023-06-15 10:45:07,664] Trial 101 finished with value: 0.06344723290926663 and parameters: {'n_estimators': 529, 'max_depth': 49, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 79 with value: 0.057266510858996617.
[I 2023-06-15 10:45:14,569] Trial 102 finished with value: 0.08186758782319396 and parameters: {'n_estimators': 459, 'max_depth': 49, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 79 with value: 0.057266510858996617.
[I 2023-06-15 10:45:18,661] Trial 103 finished with value: 0.16997764189709183 and parameters: {'n_estimators': 523, 'max_depth': 45, 'min_samples_split': 15, 'min_samples_leaf': 52}. Best is trial 79 with value: 0.057266510858996617.
[I 2023-06-15 10:45:24,908] Trial 104 finished with value: 0.07119022119631553 and parameters: {'n_estimators': 408, 'max_depth': 47, 'min_samples_split': 10, 'min_samples_leaf': 1}. Best is trial 79 with value: 0.057266510858996617.
[I 2023-06-15 10:45:30,280] Trial 105 finished with value: 0.0963

[I 2023-06-15 10:49:56,942] Trial 135 finished with value: 0.08107324640038753 and parameters: {'n_estimators': 560, 'max_depth': 49, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 79 with value: 0.057266510858996617.
[I 2023-06-15 10:50:01,177] Trial 136 finished with value: 0.14601710098123463 and parameters: {'n_estimators': 487, 'max_depth': 26, 'min_samples_split': 10, 'min_samples_leaf': 28}. Best is trial 79 with value: 0.057266510858996617.
[I 2023-06-15 10:50:07,167] Trial 137 finished with value: 0.09656474482467542 and parameters: {'n_estimators': 454, 'max_depth': 24, 'min_samples_split': 5, 'min_samples_leaf': 6}. Best is trial 79 with value: 0.057266510858996617.
[I 2023-06-15 10:50:16,915] Trial 138 finished with value: 0.08512768922912538 and parameters: {'n_estimators': 544, 'max_depth': 21, 'min_samples_split': 15, 'min_samples_leaf': 2}. Best is trial 79 with value: 0.057266510858996617.
[I 2023-06-15 10:50:27,587] Trial 139 finished with value: 0.0806

...

[I 2023-06-15 12:25:12,649] Trial 883 finished with value: 0.08262410886287969 and parameters: {'n_estimators': 79, 'max_depth': 22, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:25:17,184] Trial 884 finished with value: 0.10796678848676047 and parameters: {'n_estimators': 408, 'max_depth': 25, 'min_samples_split': 30, 'min_samples_leaf': 2}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:25:24,659] Trial 885 finished with value: 0.05806225920054443 and parameters: {'n_estimators': 452, 'max_depth': 23, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:25:32,514] Trial 886 finished with value: 0.08487768044451999 and parameters: {'n_estimators': 609, 'max_depth': 21, 'min_samples_split': 15, 'min_samples_leaf': 2}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:25:41,344] Trial 887 finished with value: 0.058081

[I 2023-06-15 12:29:11,971] Trial 917 finished with value: 0.09030679570180027 and parameters: {'n_estimators': 634, 'max_depth': 23, 'min_samples_split': 20, 'min_samples_leaf': 1}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:29:21,462] Trial 918 finished with value: 0.06317497096986054 and parameters: {'n_estimators': 607, 'max_depth': 19, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:29:29,674] Trial 919 finished with value: 0.08500051235721018 and parameters: {'n_estimators': 618, 'max_depth': 22, 'min_samples_split': 10, 'min_samples_leaf': 4}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:29:37,977] Trial 920 finished with value: 0.07223733565197829 and parameters: {'n_estimators': 577, 'max_depth': 21, 'min_samples_split': 5, 'min_samples_leaf': 3}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:29:42,257] Trial 921 finished with value: 0.17068

[I 2023-06-15 12:33:32,421] Trial 951 finished with value: 0.08074341659542869 and parameters: {'n_estimators': 493, 'max_depth': 25, 'min_samples_split': 10, 'min_samples_leaf': 3}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:33:41,985] Trial 952 finished with value: 0.081447245722393 and parameters: {'n_estimators': 565, 'max_depth': 21, 'min_samples_split': 15, 'min_samples_leaf': 1}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:33:46,213] Trial 953 finished with value: 0.16211723191301655 and parameters: {'n_estimators': 535, 'max_depth': 19, 'min_samples_split': 5, 'min_samples_leaf': 43}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:33:54,659] Trial 954 finished with value: 0.07098919305529383 and parameters: {'n_estimators': 508, 'max_depth': 23, 'min_samples_split': 10, 'min_samples_leaf': 1}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:34:01,508] Trial 955 finished with value: 0.07238

[I 2023-06-15 12:38:11,783] Trial 985 finished with value: 0.15296919408946286 and parameters: {'n_estimators': 583, 'max_depth': 30, 'min_samples_split': 10, 'min_samples_leaf': 34}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:38:22,130] Trial 986 finished with value: 0.08977088534218192 and parameters: {'n_estimators': 645, 'max_depth': 26, 'min_samples_split': 15, 'min_samples_leaf': 3}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:38:38,019] Trial 987 finished with value: 0.05748034925669561 and parameters: {'n_estimators': 627, 'max_depth': 29, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:38:52,116] Trial 988 finished with value: 0.06328316194482243 and parameters: {'n_estimators': 639, 'max_depth': 29, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 538 with value: 0.05705357842062288.
[I 2023-06-15 12:38:59,695] Trial 989 finished with value: 0.1343

Number of finished trials: 1000
Best trial: {'n_estimators': 342, 'max_depth': 22, 'min_samples_split': 5, 'min_samples_leaf': 1}


In [52]:
def gbr_objective(trial):
    
    param = {  
        
        'learning_rate': trial.suggest_float('learning_rate', 0.001,0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100,6000, step=50),
        'max_depth': trial.suggest_categorical('max_depth', [2,3,4,5,6,7,8,9,10]),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 46),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 26),
        'max_features': 'sqrt'
        }

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    model = GradientBoostingRegressor(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    '''model = GradientBoostingRegressor(**param)
    model.fit(X, y)
    predictions = model.predict(X)
    rmse = mean_squared_error(y, predictions)'''
    
    return rmse

In [53]:
study_gbr = optuna.create_study(sampler=TPESampler(), direction='minimize')
study_gbr.optimize(gbr_objective, n_trials=1000)
print('Number of finished trials:', len(study_gbr.trials))
print('Best trial:', study_gbr.best_trial.params)

[I 2023-06-15 12:58:10,129] A new study created in memory with name: no-name-1fabf067-71bd-4722-9fa4-f637c704e124
[I 2023-06-15 12:58:11,127] Trial 0 finished with value: 0.13399658742424206 and parameters: {'learning_rate': 0.0867335760143012, 'n_estimators': 1300, 'max_depth': 3, 'min_samples_leaf': 38, 'min_samples_split': 26}. Best is trial 0 with value: 0.13399658742424206.
[I 2023-06-15 12:58:14,092] Trial 1 finished with value: 0.12972218567090846 and parameters: {'learning_rate': 0.04222808787489795, 'n_estimators': 2700, 'max_depth': 5, 'min_samples_leaf': 26, 'min_samples_split': 8}. Best is trial 1 with value: 0.12972218567090846.
[I 2023-06-15 12:58:24,264] Trial 2 finished with value: 0.12949334468088894 and parameters: {'learning_rate': 0.010108844511490411, 'n_estimators': 5400, 'max_depth': 10, 'min_samples_leaf': 15, 'min_samples_split': 13}. Best is trial 2 with value: 0.12949334468088894.
[I 2023-06-15 12:58:25,768] Trial 3 finished with value: 0.1302056559503064 and

[I 2023-06-15 13:00:54,635] Trial 29 finished with value: 0.13015211856446807 and parameters: {'learning_rate': 0.029599061530678185, 'n_estimators': 1950, 'max_depth': 3, 'min_samples_leaf': 21, 'min_samples_split': 2}. Best is trial 27 with value: 0.1265963859211469.
[I 2023-06-15 13:00:57,104] Trial 30 finished with value: 0.12731795699322707 and parameters: {'learning_rate': 0.007667225334511553, 'n_estimators': 3150, 'max_depth': 3, 'min_samples_leaf': 14, 'min_samples_split': 8}. Best is trial 27 with value: 0.1265963859211469.
[I 2023-06-15 13:00:59,510] Trial 31 finished with value: 0.12916485844852807 and parameters: {'learning_rate': 0.007277210268712156, 'n_estimators': 3100, 'max_depth': 3, 'min_samples_leaf': 13, 'min_samples_split': 8}. Best is trial 27 with value: 0.1265963859211469.
[I 2023-06-15 13:01:01,384] Trial 32 finished with value: 0.1278312978962114 and parameters: {'learning_rate': 0.014574031469898878, 'n_estimators': 2450, 'max_depth': 3, 'min_samples_leaf':

[I 2023-06-15 13:02:06,208] Trial 58 finished with value: 0.14146543362064778 and parameters: {'learning_rate': 0.001154562609248034, 'n_estimators': 2750, 'max_depth': 5, 'min_samples_leaf': 28, 'min_samples_split': 21}. Best is trial 45 with value: 0.1243564753888017.
[I 2023-06-15 13:02:11,452] Trial 59 finished with value: 0.12678442919682323 and parameters: {'learning_rate': 0.0044308110490869496, 'n_estimators': 3300, 'max_depth': 10, 'min_samples_leaf': 36, 'min_samples_split': 24}. Best is trial 45 with value: 0.1243564753888017.
[I 2023-06-15 13:02:14,961] Trial 60 finished with value: 0.14292549788473105 and parameters: {'learning_rate': 0.0012624914244425268, 'n_estimators': 2400, 'max_depth': 8, 'min_samples_leaf': 33, 'min_samples_split': 19}. Best is trial 45 with value: 0.1243564753888017.
[I 2023-06-15 13:02:17,688] Trial 61 finished with value: 0.12632590642425978 and parameters: {'learning_rate': 0.011309987378987632, 'n_estimators': 2000, 'max_depth': 5, 'min_samples

[I 2023-06-15 13:04:17,089] Trial 87 finished with value: 0.12582446564323885 and parameters: {'learning_rate': 0.015182101089385008, 'n_estimators': 3500, 'max_depth': 9, 'min_samples_leaf': 3, 'min_samples_split': 26}. Best is trial 86 with value: 0.1238150807126712.
[I 2023-06-15 13:04:24,240] Trial 88 finished with value: 0.12952774984940493 and parameters: {'learning_rate': 0.013605329849241307, 'n_estimators': 3200, 'max_depth': 9, 'min_samples_leaf': 5, 'min_samples_split': 25}. Best is trial 86 with value: 0.1238150807126712.
[I 2023-06-15 13:04:30,004] Trial 89 finished with value: 0.12598654767439263 and parameters: {'learning_rate': 0.009289186326192992, 'n_estimators': 2900, 'max_depth': 9, 'min_samples_leaf': 3, 'min_samples_split': 24}. Best is trial 86 with value: 0.1238150807126712.
[I 2023-06-15 13:04:36,741] Trial 90 finished with value: 0.13031176743752115 and parameters: {'learning_rate': 0.018666151256669344, 'n_estimators': 3500, 'max_depth': 9, 'min_samples_leaf'

[I 2023-06-15 13:06:35,510] Trial 116 finished with value: 0.12619767775606675 and parameters: {'learning_rate': 0.014848599921241714, 'n_estimators': 2450, 'max_depth': 9, 'min_samples_leaf': 3, 'min_samples_split': 21}. Best is trial 86 with value: 0.1238150807126712.
[I 2023-06-15 13:06:39,175] Trial 117 finished with value: 0.12821021905977062 and parameters: {'learning_rate': 0.016327559245702467, 'n_estimators': 1950, 'max_depth': 9, 'min_samples_leaf': 6, 'min_samples_split': 23}. Best is trial 86 with value: 0.1238150807126712.
[I 2023-06-15 13:06:43,284] Trial 118 finished with value: 0.13047838678225376 and parameters: {'learning_rate': 0.01097034990391792, 'n_estimators': 2200, 'max_depth': 9, 'min_samples_leaf': 7, 'min_samples_split': 25}. Best is trial 86 with value: 0.1238150807126712.
[I 2023-06-15 13:06:48,784] Trial 119 finished with value: 0.12816400561546976 and parameters: {'learning_rate': 0.005158975035888282, 'n_estimators': 2700, 'max_depth': 10, 'min_samples_l

...

[I 2023-06-15 13:38:40,399] Trial 899 finished with value: 0.12425180156876058 and parameters: {'learning_rate': 0.04696474456031635, 'n_estimators': 2700, 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 25}. Best is trial 149 with value: 0.11892212137609426.
[I 2023-06-15 13:38:41,914] Trial 900 finished with value: 0.12434082789691138 and parameters: {'learning_rate': 0.06677623580364349, 'n_estimators': 2550, 'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 26}. Best is trial 149 with value: 0.11892212137609426.
[I 2023-06-15 13:38:45,274] Trial 901 finished with value: 0.1302380981186621 and parameters: {'learning_rate': 0.04400573390283201, 'n_estimators': 2850, 'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 14}. Best is trial 149 with value: 0.11892212137609426.
[I 2023-06-15 13:38:47,122] Trial 902 finished with value: 0.1260084280984317 and parameters: {'learning_rate': 0.07986402800965887, 'n_estimators': 3100, 'max_depth': 2, 'min_samples_l

[I 2023-06-15 13:39:50,856] Trial 928 finished with value: 0.12160144458709865 and parameters: {'learning_rate': 0.04797197615947346, 'n_estimators': 2850, 'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 25}. Best is trial 149 with value: 0.11892212137609426.
[I 2023-06-15 13:39:53,683] Trial 929 finished with value: 0.12630665780914296 and parameters: {'learning_rate': 0.06990955943346787, 'n_estimators': 4700, 'max_depth': 2, 'min_samples_leaf': 4, 'min_samples_split': 26}. Best is trial 149 with value: 0.11892212137609426.
[I 2023-06-15 13:39:56,402] Trial 930 finished with value: 0.1322113250590028 and parameters: {'learning_rate': 0.07924582090101123, 'n_estimators': 2750, 'max_depth': 4, 'min_samples_leaf': 5, 'min_samples_split': 24}. Best is trial 149 with value: 0.11892212137609426.
[I 2023-06-15 13:40:00,486] Trial 931 finished with value: 0.13053697844732814 and parameters: {'learning_rate': 0.09371944534355049, 'n_estimators': 2950, 'max_depth': 6, 'min_samples_

[I 2023-06-15 13:41:02,340] Trial 957 finished with value: 0.12753371537550584 and parameters: {'learning_rate': 0.07606835062344394, 'n_estimators': 2500, 'max_depth': 7, 'min_samples_leaf': 4, 'min_samples_split': 24}. Best is trial 149 with value: 0.11892212137609426.
[I 2023-06-15 13:41:04,323] Trial 958 finished with value: 0.12188648271797187 and parameters: {'learning_rate': 0.03163048485044868, 'n_estimators': 3100, 'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 25}. Best is trial 149 with value: 0.11892212137609426.
[I 2023-06-15 13:41:06,231] Trial 959 finished with value: 0.1307354325368558 and parameters: {'learning_rate': 0.041624344718872865, 'n_estimators': 2900, 'max_depth': 2, 'min_samples_leaf': 6, 'min_samples_split': 19}. Best is trial 149 with value: 0.11892212137609426.
[I 2023-06-15 13:41:08,055] Trial 960 finished with value: 0.1310737354297791 and parameters: {'learning_rate': 0.039367901635518135, 'n_estimators': 2750, 'max_depth': 2, 'min_samples

[I 2023-06-15 13:42:04,016] Trial 986 finished with value: 0.12787957551756976 and parameters: {'learning_rate': 0.019590613686009756, 'n_estimators': 2350, 'max_depth': 2, 'min_samples_leaf': 5, 'min_samples_split': 25}. Best is trial 149 with value: 0.11892212137609426.
[I 2023-06-15 13:42:05,424] Trial 987 finished with value: 0.11949052795243856 and parameters: {'learning_rate': 0.06626635793385702, 'n_estimators': 2300, 'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 24}. Best is trial 149 with value: 0.11892212137609426.
[I 2023-06-15 13:42:09,175] Trial 988 finished with value: 0.12857766079616736 and parameters: {'learning_rate': 0.06880052841863091, 'n_estimators': 2050, 'max_depth': 9, 'min_samples_leaf': 2, 'min_samples_split': 23}. Best is trial 149 with value: 0.11892212137609426.
[I 2023-06-15 13:42:12,813] Trial 989 finished with value: 0.12524447960545151 and parameters: {'learning_rate': 0.06606868172557762, 'n_estimators': 2100, 'max_depth': 7, 'min_sample

Number of finished trials: 1000
Best trial: {'learning_rate': 0.05005099614548874, 'n_estimators': 2200, 'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 24}
